In [103]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

#Normalize 
df = pd.read_json("train.json")
df["num_photos"] = df["photos"].apply(len)
df["num_features"] = df["features"].apply(len)
df["num_description_words"] = df["description"].apply(lambda x: len(x.split(" ")))
df["created"] = pd.to_datetime(df["created"])
df["created_year"] = df["created"].dt.year
df["created_month"] = df["created"].dt.month
df["created_day"] = df["created"].dt.day

features_to_use = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
                   "num_photos", "num_features", "num_description_words",
                   "created_year", "created_month", "created_day"]

In [91]:
lbl = preprocessing.LabelEncoder()
lbl.fit(list(df['manager_id'].values))
df['manager_id'] = lbl.transform(list(df['manager_id'].values))

In [104]:
#Add manager_id to features
features_to_use.extend(['manager_id'])
rental = df[features_to_use]
#rental.to_csv('initial.csv')
print(rental.head())

        bathrooms  bedrooms  latitude  longitude  price  num_photos  \
10            1.5         3   40.7145   -73.9425   3000           5   
10000         1.0         2   40.7947   -73.9667   5465          11   
100004        1.0         1   40.7388   -74.0018   2850           8   
100007        1.0         1   40.7539   -73.9677   3275           3   
100013        1.0         4   40.8241   -73.9493   3350           3   

        num_features  num_description_words  created_year  created_month  \
10                 0                     95          2016              6   
10000              5                      9          2016              6   
100004             4                     94          2016              4   
100007             2                     80          2016              4   
100013             1                     68          2016              4   

        created_day                        manager_id  
10               24  5ba989232d0489da1b5f2c45f6688adc  
1000

In [101]:
#Add manager_skill as a feature
X = df[features_to_use]
Y = df['interest_level']

temp = pd.concat([X.manager_id, pd.get_dummies(Y)], axis=1).groupby('manager_id').mean()
temp['count'] = X.groupby('manager_id').count().iloc[:,1]

#compute skill
temp['manager_skill'] = temp['high']*2 + temp['medium']

# get ixes for unranked managers...
unranked_managers_ixes = temp['count']<20
# ... and ranked ones
ranked_managers_ixes = ~unranked_managers_ixes

# compute mean values from ranked managers and assign them to unranked ones
mean_values = temp.loc[ranked_managers_ixes, ['high','low','medium','manager_skill']].mean()
print(mean_values)
temp.loc[unranked_managers_ixes,['high','low','medium','manager_skill']] = mean_values.values

temp['manager_id'] = temp.index
joined = pd.merge(rental, temp,on='manager_id')
joined.to_csv('rental.csv')

high             0.081314
low              0.673194
medium           0.245492
manager_skill    0.408120
dtype: float64
